In [32]:
import pandas as pd
from tqdm import tqdm
import os
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
import pyLDAvis.sklearn

tfid = TfidfVectorizer()
okt = Okt()

# 데이터 다운로드

In [33]:
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [34]:
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

'C:\\Users\\User\\DataScientist\\Movie Review\\data\\review.txt'

# 학습 데이터

In [35]:
data = pd.read_csv(os.path.join("./data", 'review.txt'), sep='\t')

In [36]:
review_data = data['document'].dropna().values

review_data = review_data[:5000]

In [37]:
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 1:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:15<00:00, 314.11it/s]


In [38]:
review_tfid = tfid.fit_transform(cleaned_review_data)

In [39]:
lda = LatentDirichletAllocation(n_components=3)  

lda.fit(review_tfid)

LatentDirichletAllocation(n_components=3)

In [40]:
labels = data['label'].iloc[:5000].values

In [41]:
lr = LogisticRegression()

lr.fit(review_tfid, labels)

LogisticRegression()

In [42]:
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\User\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


# 크롤링 데이터

In [43]:
df = pd.read_csv("naver_review.csv")

sample = df['review'].iloc[:len(df)]

cleaned_check_review_data = []

for review in tqdm(sample):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 1:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_check_review = " ".join(cleaned_tokens)
    
    cleaned_check_review_data.append(cleaned_check_review)
    


100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [00:03<00:00, 22.66it/s]


In [54]:
positive = 0
negative = 0

test_review_tfid = tfid.transform(cleaned_check_review_data)
for i in range(len(cleaned_check_review_data)):
    
    test_pred = lr.predict(test_review_tfid)[i]
    if test_pred > 0 :
        positive += 1 
    else:
        negative += 1


In [50]:
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, test_review_tfid, tfid)
pyLDAvis.display(visualization)

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\User\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [51]:
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if positive > negative else "부정"))

분석 결과 부정적인 리뷰로 예측됩니다. 


In [52]:
print(positive)
print(negative)

23
48


In [53]:
df

,Unnamed: 0,아바타: 물의 길 : 네이버 영화,user,review
0,0,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...",insu****,"\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티..."
1,1,2014년에 이글을 읽는사람들에게..,xnzm****,"\n2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n..."
2,2,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",alwa****,\n영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대...
3,3,"아바타2 3,4 제작소식 모음",soqu****,\n \n \n \n<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n...
4,4,아바타2 스토리,doub****,\n전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리...
...,...,...,...,...
66,66,"제임스 카메론 감독, <아바타 2>의 촬영이 마무리되었음을 밝히다.",c106****,\n 많은 분들의 기대(와 우려)를 한 몸에 받고 있는 <아바타 2>의 촬영...
67,67,"제임스 카메론 감독, <아바타 2>에서 판도라 행성의 사실적인 수중 세계를 보게 될...",c106****,\n <아바타> 시퀄 제작에 열중하고 있는 제임스 카메론 감독이 <아바타 2...
68,68,아바타 2 - 2020년 12월 개봉 소식 = 2020년 개봉예정영화,inee****,"\n아바타 2 (Avatar 2, 2020)\n\n아바타 2\n\n감독\n제임스 카..."
69,69,[BIFF]사회적 메시지 전하는 게 영화인이 할 일,thed****,"\n2009년 12월 개봉해 국내에서 1,333만 명의 관객을 동원한 영화 <아바타..."
